In [100]:
import numpy as np
import torch 
import pyro
import re
import pandas as pd

import importlib
import time
import pickle

from granch_utils import init_model_tensor, main_sim_tensor, lesioned_sim, compute_prob_tensor, init_params_tensor, num_stab_help
#importlib.reload(granch_utils)
importlib.reload(num_stab_help)
importlib.reload(init_model_tensor)
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)

<module 'granch_utils.lesioned_sim' from '/Users/caoanjie/Desktop/projects/looking_time_models/02_pyGRANCH/granch_utils/lesioned_sim.py'>

In [101]:
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(compute_prob_tensor)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

BATCH_INFO = {
    "jitter_n": 1, 
    "total_batch_n": 1, 
    "jitter_mode": "sampling"
}

GRID_INFO = {
    "grid_mu_start": -4, "grid_mu_end": 4, "grid_mu_step": 20, 
    "grid_sigma_start": 0.001, "grid_sigma_end": 1.8, "grid_sigma_step": 20, 
    "grid_y_start": -4, "grid_y_end": 4, "grid_y_step": 20, 
    "grid_epsilon_start": 0.000001, "grid_epsilon_end": 1, "grid_epsilon_step": 20, 
    "hypothetical_obs_grid_n": 10
}


BATCH_GRID_INFO = num_stab_help.get_batch_grid(BATCH_INFO, GRID_INFO)

PRIOR_INFO = {
    "mu_prior": 0,  
    "V_prior": 1, 
    "alpha_prior": 3, 
    "beta_prior": 1, 
    "epsilon": 0.0001, "mu_epsilon":0.0001, "sd_epsilon": 0.0001, 
    "hypothetical_obs_grid_n": 10, 
    "world_EIGs": 0.0001, "max_observation": 500
}

tensor_stimuli = num_stab_help.sample_spore_experiment(pair_each_stim = 1, n_feature=3)

tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], tensor_stimuli[0])

params = init_params_tensor.granch_params(
                grid_mu =  BATCH_GRID_INFO["grid_mus"][0].to(device),
                grid_sigma = BATCH_GRID_INFO["grid_sigmas"][0].to(device),
                grid_y = BATCH_GRID_INFO["grid_ys"][0].to(device),
                grid_epsilon = BATCH_GRID_INFO["grid_epsilons"][0].to(device),
                hypothetical_obs_grid_n = PRIOR_INFO["hypothetical_obs_grid_n"], 
                mu_prior = PRIOR_INFO["mu_prior"],
                V_prior = PRIOR_INFO["V_prior"], 
                alpha_prior = PRIOR_INFO["alpha_prior"], 
                beta_prior = PRIOR_INFO["beta_prior"],
                epsilon  = PRIOR_INFO["epsilon"], 
                mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                world_EIGs = PRIOR_INFO["world_EIGs"],
                max_observation = PRIOR_INFO["max_observation"], 
                forced_exposure_max= np.nan)
        
            # add the various different cached bits
params.add_meshed_grid()
params.add_lp_mu_sigma()
params.add_y_given_mu_sigma()
params.add_lp_epsilon()
params.add_priors()

res = main_sim_tensor.granch_main_simulation(params, tensor_model, tensor_stimuli[0])
#res = lesioned_sim.granch_no_learning_simulation(params, tensor_model, tensor_stimuli[0])
#res = lesioned_sim.granch_no_noise_simulation(params, tensor_model, tensor_stimuli[0])
pd.set_option('display.max_rows', None)

#print(res.behavior)
#main_sim_tensor.granch_main_simulation(PRIOR_INFO, tensor_model, tensor_stimuli)


tensor([[ 0.1370, -0.0549, -0.0583],
        [ 0.1370, -0.0549, -0.0361],
        [ 0.1370, -0.0549, -0.0139],
        ...,
        [ 0.3370,  0.1451,  0.0973],
        [ 0.3370,  0.1451,  0.1195],
        [ 0.3370,  0.1451,  0.1417]])


/Users/caoanjie/Desktop/projects/looking_time_models/02_pyGRANCH/granch_utils/init_model_tensor.py:292: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.23718131222523356' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  
/Users/caoanjie/Desktop/projects/looking_time_models/02_pyGRANCH/granch_utils/init_model_tensor.py:292: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.04527011282577964' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  
/Users/caoanjie/Desktop/projects/looking_time_models/02_pyGRANCH/granch_utils/init_model_tensor.py:292: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.04173522565364221' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  


RuntimeError: Tensors must have same number of dimensions: got 2 and 3

In [98]:
test_stim =  tensor_stimuli[0].stimuli_sequence
noise_epsilon = 0.01
hypothetical_obs_grid_n = 3
#torch.linspace((test_stim[0] - noise_epsilon), (test_stim[0] + noise_epsilon, 3))
        

import torch

a = torch.tensor([0.1680, -0.0684, 1], dtype=torch.float64)

num_points = 10

# Create a list of tensors, each containing the linspace for one dimension
expanded_tensors = [torch.linspace(val - 0.1, val + 0.1, num_points) for val in a]

# Stack the list of tensors to get the final expanded tensor
expanded_tensor = torch.stack(expanded_tensors, dim=1).t()


d1 = expanded_tensor[0]
d2 = expanded_tensor[1]
d3 = expanded_tensor[2]

grid_1, grid_2, grid_3= torch.meshgrid(d1, d2, d3)

# Stack the grids to form tensor C
all_possible_obs = torch.stack((grid_1, grid_2, grid_3), dim=-1).view(-1, 3)

all_possible_obs

tensor([[ 0.0680, -0.1684,  0.9000],
        [ 0.0680, -0.1684,  0.9222],
        [ 0.0680, -0.1684,  0.9444],
        ...,
        [ 0.2680,  0.0316,  1.0556],
        [ 0.2680,  0.0316,  1.0778],
        [ 0.2680,  0.0316,  1.1000]])

KeyboardInterrupt: 

In [77]:
import torch

# Create tensors A and B
A = torch.tensor([1, 2, 3])
B = torch.tensor([4, 5, 6])
C = torch.tensor([7, 8, 9])

# Create all combinations using torch.meshgrid()
grid_A, grid_B, grid_C= torch.meshgrid(A,B,C)

# Stack the grids to form tensor C
D = torch.stack((grid_A, grid_B, grid_C), dim=-1)

#print(D)

# Reshape tensor C to 2D if needed
D = D.view(-1, 3)

# Print the resulting tensor C
print(D.size())


torch.Size([27, 3])


RuntimeError: torch.meshgrid: Expected 0D or 1D tensor in the tensor list but got:  3  5
 3  6
 4  5
 4  6
[ torch.LongTensor{4,2} ]